# Nash Q-Learning
This tutorial  demonstrates Nash Q-Learning on a zero-sum strategic (non-extensive) game. 

For the payoff matrix, row player is maximizer, coloumn player is minimizer.

Reference: 

Status: TODO

In [3]:
import numpy as np
! pip install pandas
import pandas as pd
# GameMatrix = np.array([[-2,3], [3,-4]])
GameMatrix = np.array([[0,2,-1], [-1,0,1], [1,-1,0]])

Itr = 10000
print('Game matrix: ')
pd.DataFrame(GameMatrix)


Game matrix: 


,0,1,2
0,0,2,-1
1,-1,0,1
2,1,-1,0


Reward matrices: $[t, s, (a_1, a_2), s'] -> r$

Transition probability matrices: $[t, s, (a_1, a_2), s'] -> p$

In [1]:
import sys
sys.path.append("../..")
from mars.env.mdp import ArbitraryMDP, MDPWrapper
import numpy as np

num_states = 2
num_actions_per_player = 2
num_trans = 2

env = MDPWrapper(ArbitraryMDP(num_states=2, num_actions_per_player=2, num_trans=2))
trans_matrices = env.env.trans_prob_matrices # shape: [dim_transition, dim_state, dim_action (p1*p2), dim_state]
reward_matrices = env.env.trans_prob_matrices # shape: [dim_transition, dim_state, dim_action (p1*p2), dim_state]

_, oracle_nash_q, _ = env.NEsolver(verbose=True)
print(env.observation_space, env.action_space)
# env.render()
obs = env.reset()
print(obs)
done = False
while not np.any(done):
    obs, r, done, _ = env.step([1,0])
    print(obs, r, done)


Nash values of all states (from start to end): 
 [[-0.0358505795825428, -0.030080603952543233], [-0.29794753846881733, -0.16877784345195818]]
Nash Q-values of all states (from start to end): 
 [array([[[-0.08483362,  0.20379178],
        [ 0.17264495, -1.05588438]],

       [[ 0.56382894, -0.0300806 ],
        [ 0.45040832, -0.26029949]]]), array([[[-2.97947538e-01,  6.24537257e-01],
        [-3.04179494e-01, -6.60926919e-01]],

       [[-5.57291825e-04, -5.91149472e-01],
        [-1.91389173e-01, -1.12004852e-01]]])]
Nash strategies of all states (from start to end): 
 [[(array([0.80975876, 0.19024124]), array([0.83028852, 0.16971148])), (array([1.00000000e+00, 2.90380966e-11]), array([9.42400068e-12, 1.00000000e+00]))], [(array([9.99999996e-01, 4.49219986e-09]), array([1.00000000e+00, 6.29184839e-11])), (array([0.11848822, 0.88151178]), array([0.71516631, 0.28483369]))]]
Box(0.0, 6.0, (1,), float32) Discrete(2)
[[1], [1]]
[[3], [3]] [0.8893378340991678, -0.8893378340991678] [False, F

/home/quantumiracle/anaconda3/envs/res/lib/python3.6/site-packages/ecos/ecos.py:26: UserWarning: Converting G to a CSC matrix; may take a while.
  warn("Converting G to a CSC matrix; may take a while.")
/home/quantumiracle/anaconda3/envs/res/lib/python3.6/site-packages/ecos/ecos.py:29: UserWarning: Converting A to a CSC matrix; may take a while.
  warn("Converting A to a CSC matrix; may take a while.")


In [8]:
# initialize Nash Q table 
nash_q_tab = np.ones(((num_trans+1)*num_states, num_actions_per_player, num_actions_per_player))  # there is (num_trans+1) state space
print(nash_q_tab.shape)
print(np.array(oracle_nash_q).shape)

(6, 2, 2)
(2, 2, 2, 2)


In [24]:
from mars.env.mdp.utils.nash_solver import NashEquilibriumECOSSolver

gamma = 1
tau = 0.1
itr = 1

def get_matrix(q_tab, state):
    return q_tab[state]

def get_nash_value(payoff):
    ne = NashEquilibriumECOSSolver(payoff)
    ne_value = ne[0]@payoff@ne[1].T
    
    return ne_value

def get_random_action(dim):
    return np.random.randint(0, dim, size=2)

def compare_with_oracle(nash_q, oracle_nash_q):
    nash_q_ = nash_q[:-num_states]  # no nash value for last-step states
    nash_q_ = nash_q_.reshape(num_trans, num_states, num_actions_per_player, num_actions_per_player)

    mse = np.mean((nash_q_ - np.array(oracle_nash_q))**2)
    return mse

for i in range(itr):
    mse = compare_with_oracle(nash_q_tab, oracle_nash_q)
    print("mse: ", mse)
    obs = env.reset()
    done = False
    while not np.any(done):
        # action = [1,0]
        action = get_random_action(num_actions_per_player)
        next_obs, r, done, _ = env.step(action)

        next_obs_payoff = get_matrix(nash_q_tab, next_obs[0][0])
        next_obs_nash_value = get_nash_value(next_obs_payoff)

        nash_target = r[0]+gamma*next_obs_nash_value  # from agent 0's perspective

        nash_q_tab[obs[0][0], action[0], action[1]] = tau*nash_target + (1-tau)*nash_q_tab[obs[0][0], action[0], action[1]]
        
# print(nash_q_tab)


mse:  1.345663107602347
mse:  1.33613935719146
mse:  1.3292929931389326
mse:  1.3465952335098228
mse:  1.3624234127060286
mse:  1.3597438037806189
mse:  1.351258747990542
mse:  1.3492423955971882
mse:  1.35386392293633
mse:  1.3603819465018072
mse:  1.3666797551187604
mse:  1.366104986942699
mse:  1.3677418465502575
mse:  1.3513664618726713
mse:  1.3522259609611105
mse:  1.3606767390130496
mse:  1.3581747914568296
mse:  1.3743523132880429
mse:  1.3688989610866225
mse:  1.3641529081241022
mse:  1.3673814328978104
mse:  1.3627735813438455
mse:  1.3678882435609931
mse:  1.3681469015130683
mse:  1.3821753371724799
mse:  1.3883150231613102
mse:  1.3865101286265464
mse:  1.3851704609443005
mse:  1.3866203670867094
mse:  1.3808809135422377
mse:  1.3679576616829165
mse:  1.3629835686468166
mse:  1.3694758557654652
mse:  1.364968154608257
mse:  1.3535773135299025
mse:  1.3436005962417068
mse:  1.3447709812561532
mse:  1.3488090338014171
mse:  1.3349143585252474
mse:  1.323549602609321
mse:  1.3